In [1]:
cd ..

/Users/wliao0504/Library/Mobile Documents/com~apple~CloudDocs/Repo/hca-lab/CLIF-MIMIC


/Users/wliao0504/.pyenv/versions/3.13.0/lib/python3.13/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# src/tables/medication_admin_continuous.py
import numpy as np
import pandas as pd
import logging
from importlib import reload
import src.utils
reload(src.utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, search_mimic_items
from fuzzywuzzy import process

loaded configuration from config.json
loaded configuration from config.json


In [3]:
setup_logging()

MAC_COLUMNS = [
    "hospitalization_id", "med_order_id", "admin_dttm", "med_name", "med_category", "med_group", 
    "med_route_name", "med_route_category", "med_dose", "med_dose_unit", "mar_action_name", "mar_action_category"
]

MAC_COL_RENAME_MAPPER = {
    "dose": "med_dose",
    "rateuom": "med_dose_unit",
    "amountuom": "med_dose_unit",
    "new_mar": "mar_action_name", 
    "linkorderid": "med_order_id",
    "recorded_dttm": "admin_dttm"
}

MAC_MCIDE_URL = "https://raw.githubusercontent.com/clif-consortium/CLIF/main/mCIDE/clif_medication_admin_continuous_med_categories.csv"

def map_name_to_category(name, categories):
    '''
    Map a medication name to a category using fuzzy matching.
    '''
    match, score = process.extractOne(name, categories)
    return match if score >= 80 else None

def are_doses_close(doses):
    return (abs(doses.iloc[0] - doses.iloc[1]) / max(doses.iloc[0], doses.iloc[1])) <= 0.1

# drop the row with the shorter mar_action_name
def drop_shorter_action_name(group):
    if len(group) == 2 and are_doses_close(group['med_dose']):
        return group.loc[[group['mar_action_name'].str.len().idxmax()]]
    return group

2025-01-07 02:40:53,431 - INFO - initialized logging at logs/test.log


In [4]:
mac_mapping = load_mapping_csv("mac")
mac_mapper = construct_mapper_dict(mac_mapping, "itemid", "med_category")

logging.info("parsing the mapping files to identify relevant items and fetch corresponding events...")
mac_item_ids = get_relevant_item_ids(
    mapping_df = mac_mapping, 
    decision_col = "decision", 
    excluded_labels = ["NO MAPPING", "UNSURE", "MAPPED ELSEWHERE", "NOT AVAILABLE", "TO MAP, ELSEWHERE"]
    ) 

mac_events = fetch_mimic_events(mac_item_ids)

2025-01-07 02:40:53,978 - INFO - parsing the mapping files to identify relevant items and fetch corresponding events...
2025-01-07 02:40:53,980 - INFO - querying the d_items table to identify which event tables to be separately queried for 67 items
2025-01-07 02:40:54,443 - INFO - identified 1 event tables to be separately queried: ['inputevents']
2025-01-07 02:40:54,444 - INFO - fetching events from inputevents table for 67 items
2025-01-07 02:40:54,444 - WARNING - inputevents not yet supported, thus returning all columns
2025-01-07 02:41:46,041 - INFO - fetched 4673490 events from inputevents table for 67 items
2025-01-07 02:41:46,044 - INFO - concatenated 4673490 events from 1 event tables


In [5]:
mac_events.head()


,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,originalamount,originalrate,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,10000690,25860671,37081114,8787,2150-11-06 12:00:00,2150-11-06 12:01:00,2150-11-06 13:16:00,220949,100.000000,mL,...,100.000000,0.0,Dextrose 5%,Dextrose 5%,inputevents,Fluids/Intake,mL,Solution,NaN,NaN
1,10000690,25860671,37081114,8787,2150-11-06 13:00:00,2150-11-06 13:01:00,2150-11-06 13:16:00,220949,200.000000,mL,...,200.000000,0.0,Dextrose 5%,Dextrose 5%,inputevents,Fluids/Intake,mL,Solution,NaN,NaN
2,10000690,25860671,37081114,17393,2150-11-02 20:00:00,2150-11-02 22:45:00,2150-11-02 20:14:00,221749,5.475664,mg,...,60.000000,0.6,Phenylephrine,Phenylephrine,inputevents,Medications,mg,Solution,NaN,NaN
3,10000690,25860671,37081114,17393,2150-11-02 22:45:00,2150-11-02 23:36:00,2150-11-02 23:12:00,221749,1.410112,mg,...,54.524338,0.5,Phenylephrine,Phenylephrine,inputevents,Medications,mg,Solution,NaN,NaN
4,10000690,25860671,37081114,17393,2150-11-02 23:36:00,2150-11-03 00:35:00,2150-11-03 00:42:00,221749,1.305181,mg,...,53.114223,0.4,Phenylephrine,Phenylephrine,inputevents,Medications,mg,Solution,NaN,NaN


In [6]:
# s stands for simple    
mac_events_s = mac_events[[
    'subject_id', 'hadm_id', 'starttime',
    'endtime', 'storetime', 'statusdescription', 'itemid', 'amount', 'amountuom', 'rate',
    'rateuom', # 'orderid', 
    'linkorderid', # 'ordercategoryname',
    'totalamount', 'totalamountuom', 'originalamount', 'originalrate'
    ]].reset_index(drop = True)
# mac_events_s = convert_and_sort_datetime(mac_events_s)
# RESUME HERE

In [7]:
convert_and_sort_datetime(mac_events_s)

,index,subject_id,hadm_id,starttime,endtime,storetime,statusdescription,itemid,amount,amountuom,rate,rateuom,linkorderid,totalamount,totalamountuom,originalamount,originalrate
0,0,14046553,20000094,2150-03-02 15:25:00,2150-03-02 16:48:00,2150-03-02 15:26:00,ChangeDose/Rate,221906,1.623472,mg,0.180110,mcg/kg/min,6453254,250.0,mL,8.000000,0.1800
1,1,14046553,20000094,2150-03-02 16:34:00,2150-03-02 18:42:00,2150-03-02 16:35:00,ChangeDose/Rate,220949,69.565216,mL,32.608696,mL/hour,4110913,250.0,mL,250.000000,32.6087
2,2,14046553,20000094,2150-03-02 16:34:00,2150-03-02 18:42:00,2150-03-02 16:35:00,ChangeDose/Rate,221653,69.565211,mg,5.004403,mcg/kg/min,4110913,250.0,mL,250.000000,5.0000
3,3,14046553,20000094,2150-03-02 16:48:00,2150-03-02 18:32:00,2150-03-02 17:00:00,ChangeDose/Rate,221906,1.806973,mg,0.159988,mcg/kg/min,6453254,250.0,mL,6.376528,0.1600
4,4,14046553,20000094,2150-03-02 18:32:00,2150-03-02 18:45:00,2150-03-02 18:38:00,ChangeDose/Rate,221906,0.253864,mg,0.179816,mcg/kg/min,6453254,250.0,mL,4.569556,0.1800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4673485,4673485,16572164,29999828,2180-10-28 18:51:00,2180-10-28 18:52:00,2180-10-28 18:51:00,FinishedRunning,225975,1.000000,dose,NaN,None,9213048,NaN,None,1.000000,1.0000
4673486,4673486,16572164,29999828,2180-10-29 08:00:00,2180-10-29 08:01:00,2180-10-29 14:22:00,FinishedRunning,225975,1.000000,dose,NaN,None,132195,NaN,None,1.000000,1.0000
4673487,4673487,16572164,29999828,2180-10-29 08:13:00,2180-10-29 08:14:00,2180-10-29 12:13:00,FinishedRunning,223258,2.000000,units,NaN,None,1407695,NaN,None,2.000000,2.0000
4673488,4673488,16572164,29999828,2180-10-29 12:13:00,2180-10-29 12:14:00,2180-10-29 12:13:00,FinishedRunning,223258,4.000000,units,NaN,None,7999577,NaN,None,4.000000,4.0000


In [8]:
# final check there is no dup for god's sake
mac_ldfd.duplicated(subset=meds_keycols, keep=False).sum()

SyntaxError: invalid syntax (434724452.py, line 2)